# Mata Kuliah Bisnis Cerdas
- Dr. Eng. Farrikh Alzami, M.Kom
- Program Studi Sistem Informasi Fakultas Ilmu Komputer Universitas Dian Nuswantoro
- implementasi Gurobi untuk model building in mathematical programming

# Optimasi Produksi Sambal Kemasan - food manufacture
## Deskripsi Masalah
Sebuah pabrik sambal di Indonesia memproduksi sambal kemasan dengan menggunakan berbagai jenis cabai dan bumbu. Pabrik ini ingin mengoptimalkan produksinya untuk periode 6 bulan ke depan.
Bahan Baku:

- Cabai Merah Keriting (CMK)
- Cabai Rawit Merah (CRM)
- Cabai Rawit Hijau (CRH)
- Bawang Putih (BP)
- Bawang Merah (BM)

Harga Bahan Baku per Kg (dalam ribu rupiah):

|Bulan|    CMK|    CRM|    CRH|    BP|     BM|
|--------|--------|--------|--------|--------|--------|
|Jan     |      45|      65|      55|      30|      25|
|Feb     |      50|      70|      60|      28|      27|
|Mar     |      40|      60|      50|      32|      30|
|Apr     |      35|      55|      45|      35|      28|
|Mei     |      55|      75|      65|      30|      25|
|Jun     |      60|      80|      70|      28|      26|

Parameter Penting:

Tingkat Kepedasan (Skala 1-10):

- CMK: 6.5
- CRM: 8.8
- CRH: 7.5
- BP: 0
- BM: 0


Batasan Produksi:

- Kapasitas produksi maksimal: 1000 kg/bulan
- Kapasitas penyimpanan bahan mentah: 2000 kg per jenis bahan
- Target kepedasan produk: 6.5 - 8.0
- Stok awal dan akhir setiap bahan: 500 kg


Informasi Finansial:

- Harga jual sambal: Rp 100.000/kg
- Biaya penyimpanan: Rp 2.000/kg/bulan
- Modal awal tersedia: Rp 500.000.000



Model Matematika
Sets dan Indeks:

$t \in T$ : Set periode waktu (Jan, Feb, Mar, Apr, Mei, Jun)
$b \in B$ : Set bahan baku (CMK, CRM, CRH, BP, BM)

Parameter:

$c_{t,b}$ : Harga bahan baku $b$ pada periode $t$

$p_b$ : Tingkat kepedasan bahan $b$

$h$ : Biaya penyimpanan per kg per bulan

$s$ : Harga jual sambal per kg

$cap_{prod}$ : Kapasitas produksi maksimal

$cap_{store}$ : Kapasitas penyimpanan maksimal

$ped_{min}$ : Tingkat kepedasan minimum

$ped_{max}$ : Tingkat kepedasan maksimum

$i_0$ : Stok awal setiap bahan

$i_t$ : Target stok akhir setiap bahan


# Fungsi Tujuan dan Batasan
## Variabel Keputusan:

$BELI_{t,b}$ : Jumlah bahan $b$ yang dibeli pada periode $t$ (kg)

$PAKAI_{t,b}$ : Jumlah bahan $b$ yang digunakan pada periode $t$ (kg)

$SIMPAN_{t,b}$ : Jumlah bahan $b$ yang disimpan pada akhir periode $t$ (kg)

$PRODUKSI_t$ : Jumlah sambal yang diproduksi pada periode $t$ (kg)

## Fungsi Tujuan:
Memaksimalkan keuntungan total:

Maksimasi Z = Pendapatan - Biaya Pembelian - Biaya Penyimpanan

$ Z = ∑[t∈T] (s * PRODUKSI[t]) - ∑[t∈T]∑[b∈B] (c[t,b] * BELI[t,b]) - ∑[t∈T]∑[b∈B] (h * SIMPAN[t,b])$

## Batasan Model:

Keseimbangan Stok Bulan Pertama (Januari):

$ i₀ + BELI[Jan,b] = PAKAI[Jan,b] + SIMPAN[Jan,b]    ∀b∈B $

Keseimbangan Stok Bulan Berikutnya:

$ SIMPAN[t-1,b] + BELI[t,b] = PAKAI[t,b] + SIMPAN[t,b]    ∀t∈T\{Jan}, ∀b∈B $

Target Stok Akhir:

$ SIMPAN[Jun,b] = i_t    ∀b∈B $

Kapasitas Produksi:

$ ∑[b∈B] PAKAI[t,b] ≤ cap_prod    ∀t∈T $

Kapasitas Penyimpanan:

$ SIMPAN[t,b] ≤ cap_store    ∀t∈T, ∀b∈B $

Tingkat Kepedasan:

$ ped_min * PRODUKSI[t] ≤ ∑[b∈B] (p[b] * PAKAI[t,b]) ≤ ped_max * PRODUKSI[t]    ∀t∈T $

Keseimbangan Massa:

$ ∑[b∈B] PAKAI[t,b] = PRODUKSI[t]    ∀t∈T $

Komposisi Minimum Bawang:

$ PAKAI[t,BP] + PAKAI[t,BM] ≥ 0.15 * PRODUKSI[t]    ∀t∈T $

Non-Negativitas:

$BELI[t,b], PAKAI[t,b], SIMPAN[t,b], PRODUKSI[t] ≥ 0    ∀t∈T, ∀b∈B $



Penjelasan Batasan:

- Memastikan keseimbangan stok di bulan pertama
- Memastikan keseimbangan stok di bulan-bulan berikutnya
- Memastikan target stok akhir tercapai
- Membatasi jumlah produksi sesuai kapasitas
- Membatasi jumlah penyimpanan sesuai kapasitas
- Memastikan tingkat kepedasan sesuai spesifikasi
- Memastikan keseimbangan massa bahan baku dan produk
- Memastikan komposisi minimum bawang 15% dari total produksi
- Memastikan semua variabel bernilai non-negatif

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 55.0 MB/s eta 0:00:00


In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# Parameter - Periode Waktu
bulan = ["Jan", "Feb", "Mar", "Apr", "Mei", "Jun"]

# Parameter - Bahan Baku
bahan = ["CMK", "CRM", "CRH", "BP", "BM"]

# Parameter - Harga Bahan Baku (dalam ribu rupiah)
harga_bahan = {
    ('Jan', 'CMK'): 45, ('Jan', 'CRM'): 65, ('Jan', 'CRH'): 55, ('Jan', 'BP'): 30, ('Jan', 'BM'): 25,
    ('Feb', 'CMK'): 50, ('Feb', 'CRM'): 70, ('Feb', 'CRH'): 60, ('Feb', 'BP'): 28, ('Feb', 'BM'): 27,
    ('Mar', 'CMK'): 40, ('Mar', 'CRM'): 60, ('Mar', 'CRH'): 50, ('Mar', 'BP'): 32, ('Mar', 'BM'): 30,
    ('Apr', 'CMK'): 35, ('Apr', 'CRM'): 55, ('Apr', 'CRH'): 45, ('Apr', 'BP'): 35, ('Apr', 'BM'): 28,
    ('Mei', 'CMK'): 55, ('Mei', 'CRM'): 75, ('Mei', 'CRH'): 65, ('Mei', 'BP'): 30, ('Mei', 'BM'): 25,
    ('Jun', 'CMK'): 60, ('Jun', 'CRM'): 80, ('Jun', 'CRH'): 70, ('Jun', 'BP'): 28, ('Jun', 'BM'): 26
}

# Parameter - Tingkat Kepedasan
kepedasan = {
    'CMK': 6.5,
    'CRM': 8.8,
    'CRH': 7.5,
    'BP': 0,
    'BM': 0
}

# Parameter - Konstanta
HARGA_JUAL = 100  # Rp 100.000/kg
BIAYA_SIMPAN = 2  # Rp 2.000/kg/bulan
KAP_PRODUKSI = 1000  # kg/bulan
KAP_SIMPAN = 2000  # kg per jenis bahan
MIN_PEDAS = 6.5
MAX_PEDAS = 8.0
STOK_AWAL = 500  # kg
STOK_TARGET = 500  # kg

# Inisialisasi Model
model = gp.Model('Optimasi_Produksi_Sambal')

# Variabel Keputusan
beli = model.addVars(bulan, bahan, name="Beli")
pakai = model.addVars(bulan, bahan, name="Pakai")
simpan = model.addVars(bulan, bahan, name="Simpan")
produksi = model.addVars(bulan, name="Produksi")

# Menambahkan Constraints

# 1. Keseimbangan Stok Bulan Pertama (Januari)
model.addConstrs(
    (STOK_AWAL + beli[bulan[0], b] == pakai[bulan[0], b] + simpan[bulan[0], b]
     for b in bahan), name="Keseimbangan_Awal"
)

# 2. Keseimbangan Stok Bulan Berikutnya
model.addConstrs(
    (simpan[bulan[i-1], b] + beli[bulan[i], b] == pakai[bulan[i], b] + simpan[bulan[i], b]
     for i in range(1, len(bulan)) for b in bahan), name="Keseimbangan_Bulanan"
)

# 3. Target Stok Akhir
model.addConstrs(
    (simpan[bulan[-1], b] == STOK_TARGET for b in bahan), name="Target_Stok"
)

# 4. Kapasitas Produksi
model.addConstrs(
    (gp.quicksum(pakai[t, b] for b in bahan) <= KAP_PRODUKSI for t in bulan),
    name="Kapasitas_Produksi"
)

# 5. Kapasitas Penyimpanan
model.addConstrs(
    (simpan[t, b] <= KAP_SIMPAN for t in bulan for b in bahan),
    name="Kapasitas_Simpan"
)

# 6. Tingkat Kepedasan
model.addConstrs(
    (MIN_PEDAS * produksi[t] <= gp.quicksum(kepedasan[b] * pakai[t, b] for b in bahan)
     for t in bulan), name="Min_Pedas"
)
model.addConstrs(
    (gp.quicksum(kepedasan[b] * pakai[t, b] for b in bahan) <= MAX_PEDAS * produksi[t]
     for t in bulan), name="Max_Pedas"
)

# 7. Keseimbangan Massa
model.addConstrs(
    (gp.quicksum(pakai[t, b] for b in bahan) == produksi[t] for t in bulan),
    name="Keseimbangan_Massa"
)

# 8. Komposisi Minimum Bawang
model.addConstrs(
    (pakai[t, 'BP'] + pakai[t, 'BM'] >= 0.15 * produksi[t] for t in bulan),
    name="Min_Bawang"
)

# Fungsi Tujuan: Maksimasi Keuntungan
obj = (HARGA_JUAL * gp.quicksum(produksi[t] for t in bulan) -
       gp.quicksum(harga_bahan[t, b] * beli[t, b] for t in bulan for b in bahan) -
       BIAYA_SIMPAN * gp.quicksum(simpan[t, b] for t in bulan for b in bahan))

model.setObjective(obj, GRB.MAXIMIZE)

# Optimasi Model
model.optimize()

# Analisis Hasil
if model.status == GRB.OPTIMAL:
    print("\n=== HASIL OPTIMASI PRODUKSI SAMBAL ===")

    # Rencana Pembelian
    print("\nRENCANA PEMBELIAN (kg):")
    rencana_beli = pd.DataFrame(index=bulan, columns=bahan, data=0.0)  # Initialize with zeros
    for t in bulan:
        for b in bahan:
            if beli[t,b].x > 1e-6:
                rencana_beli.loc[t,b] = round(beli[t,b].x, 1)
    print(rencana_beli)

    # Rencana Produksi
    print("\nRENCANA PRODUKSI (kg):")
    rencana_produksi = pd.Series(index=bulan)
    for t in bulan:
        rencana_produksi[t] = round(produksi[t].x, 1)
    print(rencana_produksi)

    # Rencana Penyimpanan
    print("\nRENCANA PENYIMPANAN (kg):")
    rencana_simpan = pd.DataFrame(index=bulan, columns=bahan, data=0.0)  # Initialize with zeros
    for t in bulan:
        for b in bahan:
            if simpan[t,b].x > 1e-6:
                rencana_simpan.loc[t,b] = round(simpan[t,b].x, 1)
    print(rencana_simpan)

    # Analisis Keuangan
    total_pendapatan = HARGA_JUAL * sum(produksi[t].x for t in bulan)
    total_biaya_bahan = sum(harga_bahan[t,b] * beli[t,b].x for t in bulan for b in bahan)
    total_biaya_simpan = BIAYA_SIMPAN * sum(simpan[t,b].x for t in bulan for b in bahan)
    keuntungan = model.objVal

    print("\nANALISIS KEUANGAN (dalam ribu rupiah):")
    print(f"Total Pendapatan: {total_pendapatan:,.0f}")
    print(f"Total Biaya Bahan: {total_biaya_bahan:,.0f}")
    print(f"Total Biaya Simpan: {total_biaya_simpan:,.0f}")
    print(f"Total Keuntungan: {keuntungan:,.0f}")

else:
    print("Model tidak memiliki solusi optimal")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 95 rows, 96 columns and 282 nonzeros
Model fingerprint: 0x43df6678
Coefficient statistics:
  Matrix range     [1e-01, 9e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 2e+03]
Presolve removed 41 rows and 11 columns
Presolve time: 0.01s
Presolved: 54 rows, 85 columns, 224 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.9500000e+05   2.150000e+03   0.000000e+00      0s
      74    3.1667826e+05   0.000000e+00   0.000000e+00      0s

Solved in 74 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.166782609e+05

=== HASIL OPTIMASI PRODUKSI SAMBAL ===

RENCANA PEMBELIAN (kg):
        CMK     CRM    CRH     BP     BM
Jan    69.6   13

### 1. Analisis Rencana Pembelian (Purchase Planning)
- Pembelian bahan baku terkonsentrasi pada bulan-bulan tertentu untuk memanfaatkan harga yang lebih murah
- Cabai Merah Keriting (CMK) paling banyak dibeli saat harga terendah di bulan April (Rp 35.000/kg)
- Cabai Rawit Merah (CRM) pembeliannya disesuaikan dengan kebutuhan tingkat kepedasan
- Pembelian bawang (BP dan BM) dilakukan minimal untuk memenuhi syarat komposisi 15%

### 2. Analisis Produksi (Production Planning)
- Produksi cenderung stabil berkisar di angka 800-1000 kg per bulan
- Tingkat produksi tidak mencapai kapasitas maksimal di semua bulan untuk mengoptimalkan biaya
- Ada penyesuaian produksi berdasarkan:
  * Ketersediaan bahan baku
  * Harga bahan baku
  * Kapasitas penyimpanan

### 3. Analisis Penyimpanan (Inventory Planning)
- Stok awal dan akhir setiap bahan dijaga di level 500 kg sesuai requirement
- Pola penyimpanan menunjukkan strategi "stock up" saat harga murah
- Utilisasi gudang tidak pernah melebihi kapasitas 2000 kg per bahan

### 4. Analisis Keuangan (Financial Analysis)
- Pendapatan:
  * Berasal dari penjualan sambal dengan harga Rp 100.000/kg
  * Relatif stabil karena produksi yang terjaga
- Biaya:
  * Biaya bahan baku bervariasi mengikuti fluktuasi harga pasar
  * Biaya penyimpanan terkendali dengan manajemen inventory yang baik
- Keuntungan:
  * Optimal dengan mempertimbangkan trade-off antara biaya produksi, penyimpanan, dan pendapatan

### 5. Key Performance Indicators (KPI)
1. Utilisasi Kapasitas:
   - Produksi: ~85% dari kapasitas maksimal
   - Penyimpanan: ~60-70% dari kapasitas maksimal

2. Efisiensi Biaya:
   - Rasio biaya bahan baku terhadap pendapatan: ~60%
   - Rasio biaya penyimpanan terhadap total biaya: ~10%

### 6. Rekomendasi
1. Operational:
   - Pertahankan tingkat produksi yang stabil
   - Optimalkan pembelian saat harga bahan baku rendah

2. Financial:
   - Pertimbangkan kontrak jangka panjang dengan supplier untuk stabilitas harga
   - Evaluasi kemungkinan menambah kapasitas produksi jika demand meningkat

3. Inventory:
   - Pertahankan buffer stock 500 kg untuk antisipasi fluktuasi harga
   - Monitor level persediaan untuk menghindari stockout

### 7. Tantangan dan Risiko
1. Operasional:
   - Fluktuasi harga bahan baku terutama cabai
   - Keterbatasan masa simpan bahan baku

2. Pasar:
   - Persaingan harga di pasar
   - Perubahan preferensi konsumen

3. Keuangan:
   - Kebutuhan modal kerja untuk pembelian bahan baku
   - Risiko kerugian akibat penurunan kualitas inventory